# Load and install package

In [38]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
import pandas as pd
import json, os, csv, re

In [40]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 200)

# Process and Analysis

In [41]:
df_comments = pd.read_json('data/initial-data/scraping/comments_by_classes.json')
df_feedback = pd.read_json('data/initial-data/scraping/feedback_by_classes.json')
manual_collecting = pd.read_csv('data/results/engineering-1.csv')

In [42]:
df_comments

,user_statement,judul,rating,type,kategori,pusat_inovasi,status,tema
0,"Permisi, Bapak/Ibu. Mohon maaf sebelumnya,\nSaya ingin memberitahu untuk akses e-Book yang tersedia di website tidak tampil secara penuh, maksudnya ada beberapa bagian kata/kalimat yang kepotong. Jadi ada hambatan untuk membaca materinya secara keseluruhan, walaupun sudah dijelaskan di beberapa video tersebut.\nTerima kasih Bapak/Ibu, mohon maaf apabila ada kesalahan kata.",Kelas Online Pendalaman Manajemen Pelayanan Publik,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
1,"perencanaan yang disusun dalam pemerintah daerah sesuai dan sangat tepat sasaran, namun pelaksanaan dari perencanaantersebet sering kali tidak sesuai dengan sasaran dan terkadang anggaran yang tersedia tidaktepat sasaran sehingga hasil yang didapat tidak sesuai dengan perencanaan",Kelas Online Memahami Reformasi Birokrasi di Indonesia,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
2,Perlunya penguatan internalisasi RB diseluruh ASN,Kelas Online Memahami Reformasi Birokrasi di Indonesia,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
3,alhamdulillah,Kelas Online Memahami Reformasi Birokrasi di Indonesia,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
4,apa dampak jika dynamic governance tidak diimplementasikan dengan baik?,Kelas Online Memahami Reformasi Birokrasi di Indonesia,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
5,Bagaimana menerapkan Reformasi Birokrasi yang sangat kompleks dan dipenuhi oleh SDM yang kurang berkualitas,Kelas Online Memahami Reformasi Birokrasi di Indonesia,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
6,Paradigma administrasi apa yang saat ini ada di Indonesia? dan apa paradigma yang cocok diterapkan di Indonesia?,Kelas Online Memahami Reformasi Birokrasi di Indonesia,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
7,seberapa efektif birokrasi tematik dalam menyelesaikan masalah-masalah pada ke 5 isu tersebut?,Kelas Online Memahami Reformasi Birokrasi di Indonesia,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
8,Apa dasar hukum dari materi dalam kelas pembelajaran ini?,Kelas Online Memahami Reformasi Birokrasi di Indonesia,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
9,Kelas Online Memahami Konsep Strategic Planning ini sangat bermanfaat dan inspiratif bagi saya yang mengemban tugas sebagai Perencana Ahli Muda.. Terimakasih SmartID.. :),Kelas Online Memahami Konsep Strategic Planning,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema


In [43]:
def ganti_karakter_spesial(text):
    if re.fullmatch(r'[-.,!? ]+', text):
        return f"<comment only> {text.strip()}"
    return text

df_feedback['user_statement'] = df_feedback['user_statement'].astype(str).str.strip()
df_feedback['user_statement'] = df_feedback['user_statement'].apply(ganti_karakter_spesial)
df_feedback

,user_statement,rating,kategori,pusat_inovasi,status,tema,judul,type
0,<comment only> .,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Manajemen Strategis Rumah Sakit,feedback
1,mantap,4,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Konsep dasar Sistem Merit,feedback
2,<comment only> .,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Konsep dasar Sistem Merit,feedback
3,"keren, semoga ada kelas yang lain dengan narsum ini..",5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Konsep dasar Sistem Merit,feedback
4,Kelasnya seru,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Manajemen Talenta,feedback
5,<comment only> -,4,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Manajemen Talenta,feedback
6,OK,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Manajemen Talenta,feedback
7,<comment only> .,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Manajemen Talenta,feedback
8,<comment only> .,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Local government finance issues: lessons and implications from the case of Korea,feedback
9,terbaik,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Pendalaman Manajemen Pelayanan Publik,feedback


In [44]:
df_comments.to_csv('data/results/scrapping-comments.csv', index=False)
df_feedback.to_csv('data/results/scrapping-feedback.csv', index=False)

In [45]:
new_df = pd.concat([df_comments, df_feedback], axis=0)
new_df.to_csv('data/results/combine-feedback-comments.csv', index=False)

In [46]:
def read_csv(filename):
    with open(filename, 'r', encoding='utf-16', errors='ignore') as f:
        reader = csv.reader(f)
        rows = list(reader)  

    header = rows[0]
    data = rows[1:]

    data_clean = [row for row in data if len(row) == len(header)]
    df = pd.DataFrame(data_clean, columns=header)
    return df

In [47]:
review_users = os.listdir('data/initial-data/review-playstore')
df = pd.DataFrame({})

for review in review_users:
    temp = read_csv(f'data/initial-data/review-playstore/{review}')
    df = pd.concat([df, temp], axis=0)

In [48]:
df_review = df[['Star Rating', 'Review Text']].copy()
df_review = df_review.rename(columns={'Star Rating': 'rating', 'Review Text': 'user_statement'})
df_review['user_statement'] = df_review['user_statement'].astype(str).str.strip()
df_review['user_statement'] = df_review['user_statement'].replace('', pd.NA)
df_review['type'] = 'User Review Playstore'
df_review['user_statement'] = df_review['user_statement'].fillna('<no comment>')
df_review = df_review.reset_index(drop=True)

df_review

,rating,user_statement,type
0,5,Siap. Semoga bermanfaat,User Review Playstore
1,5,good apps for learning good governance,User Review Playstore
2,5,"aplikasinya bagus sekali, sangat mudah penggunaannya, dan sangat membantu👍🏻",User Review Playstore
3,5,"Banyak fitur dan kelas menarik, aplikasinya sangat bermanfaat 🙏💯",User Review Playstore
4,5,"Keren aplikasinya, sangat bermanfaat",User Review Playstore
5,5,Mantap dan membantu,User Review Playstore
6,5,<no comment>,User Review Playstore
7,5,<no comment>,User Review Playstore
8,5,Aplikasi yang sabgat bermanfaat untuk ASN,User Review Playstore
9,5,"Aplikasinya sangat bermanfaat buat ASN, terimakasih smartid",User Review Playstore


In [49]:
df_join = pd.concat([df_review, new_df], axis=0)
df_join = df_join.drop(['judul', 'kategori', 'pusat_inovasi', 'status', 'tema'], axis=1)
df_join

,rating,user_statement,type
0,5,Siap. Semoga bermanfaat,User Review Playstore
1,5,good apps for learning good governance,User Review Playstore
2,5,"aplikasinya bagus sekali, sangat mudah penggunaannya, dan sangat membantu👍🏻",User Review Playstore
3,5,"Banyak fitur dan kelas menarik, aplikasinya sangat bermanfaat 🙏💯",User Review Playstore
4,5,"Keren aplikasinya, sangat bermanfaat",User Review Playstore
...,...,...,...
74,4,<comment only> -,feedback
75,4,luar biasa,feedback
76,5,banyak soal yang terputus,feedback
77,2,wah,feedback


In [50]:
df_join.to_csv('data/results/combine-base.csv', index=False)

In [51]:
df_join.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136 entries, 0 to 78
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   rating          136 non-null    object
 1   user_statement  136 non-null    object
 2   type            136 non-null    object
dtypes: object(3)
memory usage: 4.2+ KB


In [52]:
df_links = pd.read_json('data/initial-data/scraping/overal_link_class.json')
df_links

,judul,url
0,Kelas Online Manajemen Strategis Rumah Sakit,https://smartidapp.co.id/admin/kelas/detail/index/588def1e2d4f7e18ed345d2376d70d475f99b9c9
1,Kelas Online Memahami Konsep dasar Sistem Merit,https://smartidapp.co.id/admin/kelas/detail/index/ddefa6e3c1a0a40ab8a08a3ed11a45234ded9b48
2,Kelas Online Memahami Manajemen Talenta,https://smartidapp.co.id/admin/kelas/detail/index/bf5cd3c2bad2c3c04cdf9e2f8f374e60a9a04474
3,Local government finance issues: lessons and implications from the case of Korea,https://smartidapp.co.id/admin/kelas/detail/index/50f0e107ee14efa04c94db8e2c77e3aa5c722325
4,Kelas Online Pendalaman Manajemen Pelayanan Publik,https://smartidapp.co.id/admin/kelas/detail/index/f07862522e9585ffd7ca6a6138ec262480b0d91f
5,Policy science: lessons and implications from the case of Korea digital strategy,https://smartidapp.co.id/admin/kelas/detail/index/4e221cd76714967aa2b164bc3b2be1e74e3e5320
6,Kelas Online Optimalisasi Pendapatan Asli Daerah,https://smartidapp.co.id/admin/kelas/detail/index/f954a14df53b9be4e301c1faf1c6d39f3e6a0eb3
7,Kelas Online Memahami Strategi Persiapan Pra Penilaian SPBE,https://smartidapp.co.id/admin/kelas/detail/index/6ca384203eff6c2a1e0c77b28b38d387d4fbdc2f
8,Kelas Online Memahami Metode Survei Kepuasan Masyarakat,https://smartidapp.co.id/admin/kelas/detail/index/914829630f7675aa4372c4eecc51df8458d91393
9,Kelas Online Memahami Reformasi Birokrasi di Indonesia,https://smartidapp.co.id/admin/kelas/detail/index/46d4f4b96a2f046db72ad9ad80c6db7b1b4a5015


In [53]:
judul = new_df['judul'].unique()
df_tidak_terdaftar = df_links[~df_links['judul'].isin(judul)]
df_tidak_terdaftar

,judul,url
7,Kelas Online Memahami Strategi Persiapan Pra Penilaian SPBE,https://smartidapp.co.id/admin/kelas/detail/index/6ca384203eff6c2a1e0c77b28b38d387d4fbdc2f
12,Kelas Online Kiat-Kiat Penyusunan Peta Potensi Investasi Daerah,https://smartidapp.co.id/admin/kelas/detail/index/3fc15624f200b507ff6cc5eb1a88c31af473c9d7
16,Kelas Online Meningkatkan Desain Inovasi Pelayanan Publik yang Berkualitas,https://smartidapp.co.id/admin/kelas/detail/index/984e898d1beebc1810fa483e8c0e43db1282c8c0
